In [23]:
import uproot
import awkward as ak
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import hist
import vector
import os
import subprocess
import gc
import pandas as pd
print("uproot version", uproot.__version__)
print("awkward version", ak.__version__)
print("numpy version", np.__version__)
print("matplotlib version", matplotlib.__version__)
print("hist version", hist.__version__)
print("vector version", vector.__version__)
print("pandas version", pd.__version__)

uproot version 5.0.12
awkward version 2.4.3
numpy version 1.25.2
matplotlib version 3.8.0
hist version 2.7.2
vector version 1.1.1
pandas version 1.5.3


In [24]:
RUNS = [290323, 290327, 290848, 291361, 291360, 291362, 290853, 290860, 291373, 290374, 290375, 291399,
               291400, 290894, 290895, 290404, 291943, 291944, 291948, 291953, 290932, 290423, 291447, 290935, 
               290425, 290427, 291451, 291453, 291976, 291982, 290456, 290458, 290459, 291482, 291485, 290975, 
               290980, 290469, 292012, 291002, 291003, 291004, 291005, 290501, 292040, 292060, 292061, 292062, 
               291041, 290539, 290540, 292075, 292077, 292080, 290549, 290553, 291590, 292106, 292108, 292109, 
               292115, 290590, 291618, 291622, 291624, 292140, 290612, 292160, 292162, 292163, 292164, 292166, 
               290632, 291657, 292168, 292192, 290658, 290660, 291690, 291692, 291694, 291698, 291706, 290687, 
               290692, 290696, 290699, 292242, 292265, 291755, 292269, 292270, 291760, 292273, 292274, 290742, 
               291769, 291263, 290764, 290766, 291283, 291284, 291285, 291795, 291796, 290776, 291803, 290787]

SAMPLE_RUNS = sorted(RUNS)
print(len(SAMPLE_RUNS))

108


In [25]:
vector.register_awkward() 
DATATYPE="mc"
assert((DATATYPE=="mc")  or (DATATYPE=="data"))
BASEDIR="/pbs/throng/training/nantes-m2-rps-exp/data" # basedir where to look for runXXX.DATATYPE.root files
IS_MC=True if DATATYPE=="mc" else False

def data_file_path(run,is_mc=IS_MC,dest=BASEDIR):
    datatype="mc" if is_mc else "data"
    print({dest},"/run",{run},".",{datatype},".root")
    return f"{dest}/run{run}.{datatype}.root" ;

#import pandas
#l1 = pandas.read_csv('../data/counters.offline.csv')['run']
#l2 = [int(e) for e in os.popen("ls /pbs/throng/training/nantes-m2-rps-exp/data/ | grep 'mc' | cut -c 4-9").read().split('\n')[0:-2]]
#print(len(l2))
#print(l2) 

In [39]:
file = uproot.open(data_file_path(SAMPLE_RUNS[0],IS_MC))
gen = file["genTree"]
gen.show()

{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {290323} . {'mc'} .root
name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
xVtxMC               | double                   | AsDtype('>f8')
yVtxMC               | double                   | AsDtype('>f8')
zVtxMC               | double                   | AsDtype('>f8')
nMuonsGen            | int32_t                  | AsDtype('>i4')
Muon_GenE            | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_GenPx           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_GenPy           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_GenPz           | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_GenLabel        | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...
Muon_GenMotherPDG... | std::vector<int32_t>     | AsJagged(AsDtype('>i4'), he...


In [29]:
def integral(histo: hist.Hist, start: float, stop: float):
    start_bin = histo.axes[0].index(start)  
    stop_bin = histo.axes[0].index(stop)
    return histo[start_bin:stop_bin].sum()

def getTracksGen(events):
    return ak.zip({
        "px":events["Muon_GenPx"],
        "py":events["Muon_GenPy"],
        "pz":events["Muon_GenPz"],
        "E":events["Muon_GenE"],
        "label":events["Muon_GenLabel"],
        "code":events["Muon_GenMotherPDGCode"]
    }, with_name='Momentum4D')


def scan(dataDescription, 
              hMagGen:hist.Hist, hPhiGen:hist.Hist, hEtaGen:hist.Hist, hPtGen:hist.Hist, hMassGen:hist.Hist, hMass2Gen:hist.Hist,
              eventSelectorGen = lambda x:[True]*len(x), trackSelectorGen = lambda x:[True]*len(x),
              verbose:bool = False):
    """ Loop over data to fill the invariant mass histogram.
        
        :param: dataDescription: is anything uproot.iterate can take.
                typical something like run*.data.root:eventsTree in our case
        :param: eventSelector: returns an array of bool from an array of events
        :param: trackSelector: returns an array of bool from an array of tracks
        print(dir(vector.backends.awkward.MomentumArray4D))
    """
    
    Entries = 0
    goodEntries = 0
    PDG = []
    
    for batch in uproot.iterate(dataDescription,
                                ["Muon_GenE", "Muon_GenPx", "Muon_GenPy", "Muon_GenPz", "Muon_GenLabel", "Muon_GenMotherPDGCode"],                                
                                report = True):

        events = batch[0] # batch[1] is the report info

        if len(events) < 1000:
            print("something is wrong",batch[1]) # this is a protection for some corrupted input data files 
            break

        goodEvents = events[eventSelectorGen(events)] 
        tracks = getTracksGen(goodEvents)
        goodTracks = tracks[trackSelectorGen(tracks)]
        #goodTracks.show()

        hMagGen.fill(ak.flatten(goodTracks.p))
        hPhiGen.fill(ak.flatten(goodTracks.phi))
        hPtGen.fill(ak.flatten(goodTracks.pt))
        hEtaGen.fill(ak.flatten(goodTracks.eta))
        hMassGen.fill(ak.flatten(goodTracks.mass))

        pairs = ak.combinations(goodTracks, 2)

        Px = pairs["0"].px + pairs["1"].px
        Py = pairs["0"].py + pairs["1"].py
        Pz = pairs["0"].pz + pairs["1"].pz
        E = pairs["0"].E + pairs["1"].E

        pairs2 = ak.zip({"px":Px, "py":Py, "pz":Pz, "E":E}, with_name = 'Momentum4D')

        hMass2Gen.fill(ak.flatten(pairs2.mass))
        
        Entries += len(events)
        goodEntries += len(ak.flatten(goodTracks))/2
        PDG = np.union1d(PDG, ak.flatten(goodTracks.code))


        if verbose:
            print("-----------------------------------------------------------")
            print(batch[1], "\nEntries: ", Entries, "\nJ/Psi: ", int(goodEntries), "\nPDG: ", PDG)
        gc.collect()
        
    return Entries, goodEntries, PDG

In [31]:
hMag = hist.Hist(hist.axis.Regular(bins = 100, start = 0, stop = 100, name = '$|p|$'))
hPhi = hist.Hist(hist.axis.Regular(bins = 200, start = -22/7, stop = 22/7, name = '$\phi$'))
hEta = hist.Hist(hist.axis.Regular(bins = 100, start = -6, stop = 0, name = '$\eta$'))
hPt = hist.Hist(hist.axis.Regular(bins = 100, start = 0, stop = 100, name = '$p_T$'))
hMass = hist.Hist(hist.axis.Regular(bins = 100, start = 0, stop = 0.2, name = '$m$'))

hMass2 = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{q \overline{q}}$'))

Entries = goodEntries = 0
PDG = []
NTotalJPsi =0
NJPsi=[]

for run in SAMPLE_RUNS:
    a, b, c = scan(dataDescription = f"/pbs/throng/training/nantes-m2-rps-exp/data/run{run}.mc.root:genTree",
                   hMagGen = hMag, hPhiGen = hPhi, hEtaGen = hEta, hPtGen = hPt, hMassGen = hMass, hMass2Gen = hMass2,
                   eventSelectorGen = lambda x: x["Muon_GenMotherPDGCode"] == 443,
                   trackSelectorGen = lambda x: (((x.eta < -2.5) & (x.eta >-4 )) & ((x.pt > 0) & (x.pt <10))),
                   verbose = True)
    Entries += a;
    goodEntries += b;
    NJPsi.append(int(b))
    PDG = np.union1d(PDG, c);
    NTotalJPsi += goodEntries
    
#print("-----------------------------------------------------------", "\nEntries: ", Entries, "\nJ/Psi: ", int(goodEntries), "\nPDG: ", PDG)
print(NTotalJPsi)
print(NJPsi)

-----------------------------------------------------------
<Report start=0 stop=70000 source='/pbs/throng/training/nantes-m2-rps-exp/data/run290323.mc.root:/genTree;1'> 
Entries:  70000 
J/Psi:  27605 
PDG:  [443]
-----------------------------------------------------------
<Report start=0 stop=6000 source='/pbs/throng/training/nantes-m2-rps-exp/data/run290327.mc.root:/genTree;1'> 
Entries:  6000 
J/Psi:  2392 
PDG:  [443]
-----------------------------------------------------------
<Report start=0 stop=40000 source='/pbs/throng/training/nantes-m2-rps-exp/data/run290374.mc.root:/genTree;1'> 
Entries:  40000 
J/Psi:  15865 
PDG:  [443]
-----------------------------------------------------------
<Report start=0 stop=20000 source='/pbs/throng/training/nantes-m2-rps-exp/data/run290375.mc.root:/genTree;1'> 
Entries:  20000 
J/Psi:  7963 
PDG:  [443]
-----------------------------------------------------------
<Report start=0 stop=80000 source='/pbs/throng/training/nantes-m2-rps-exp/data/run29

In [35]:
print(NTotalJPsi)
"""[27605, 2392, 15865, 7963, 31600, 11925, 31617, 11835, 7974, 7950, 
 59373, 35589, 43442, 19867, 15811, 27604, 19785, 31754, 11870, 7966, 
 2788, 31622, 31745, 43718, 19875, 15851, 27720, 11865, 15833, 15962, 
 15871, 35779, 19702, 59231, 11872, 3934, 19861, 11942, 27659, 7966, 
 39452, 27649, 39472, 11911, 47554, 59427, 15857, 27836, 8008, 35609, 
 7894, 27723, 7822, 15827, 15776, 19655, 15596, 51591, 31667, 7945, 
 27669, 15759, 11831, 23761, 19727, 7826, 15938, 15856, 15854, 27624, 
 23833, 797, 27606, 3968, 8005, 11855, 27682, 19865, 35700, 51493, 
 27744, 19759, 23787, 31621, 23832, 15926, 47434, 27632, 11840, 19890, 
 15915, 15846, 11795, 39560, 2771, 51299, 3231, 19768, 3198, 15852, 
 23844, 3224, 11879, 11797, 3551, 15789, 39621, 15981]"""

130401865.5


'[27605, 2392, 15865, 7963, 31600, 11925, 31617, 11835, 7974, 7950, \n 59373, 35589, 43442, 19867, 15811, 27604, 19785, 31754, 11870, 7966, \n 2788, 31622, 31745, 43718, 19875, 15851, 27720, 11865, 15833, 15962, \n 15871, 35779, 19702, 59231, 11872, 3934, 19861, 11942, 27659, 7966, \n 39452, 27649, 39472, 11911, 47554, 59427, 15857, 27836, 8008, 35609, \n 7894, 27723, 7822, 15827, 15776, 19655, 15596, 51591, 31667, 7945, \n 27669, 15759, 11831, 23761, 19727, 7826, 15938, 15856, 15854, 27624, \n 23833, 797, 27606, 3968, 8005, 11855, 27682, 19865, 35700, 51493, \n 27744, 19759, 23787, 31621, 23832, 15926, 47434, 27632, 11840, 19890, \n 15915, 15846, 11795, 39560, 2771, 51299, 3231, 19768, 3198, 15852, \n 23844, 3224, 11879, 11797, 3551, 15789, 39621, 15981]'

NameError: name 'g' is not defined

In [ ]:
### Sur un run ###
nofTracks=0 #
nofEvents=0 # 
nofJPsiGen=0
nofJPsi2SGen=0
for event in g :
    nofEvents+=1
    tracks = event["Muon"].to_list()
    for t in tracks:
        nofTracks+=1
        if t["GenMotherPDGCode"]==443 :
            nofJPsiGen +=1
        if t["GenMotherPDGCode"]== 100443 :
            nofJPsi2SGen +=1

print(nofEvents, nofTracks, nofJPsiGen/2, nofJPsi2SGen/2)
print(nofJPsiGen/2 + nofJPsi2SGen/2)

In [ ]:


#    plt.hist(g.nMuonsGen,bins=[0,1,2,3,4,5,6,7,8,9,10]) #nbr muons par event
 #   plt.yscale("log")



